In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import numpy as np
from collections import defaultdict

In [3]:
from mnist_loader import *

from train import train
from vae import VariationalAutoEncoder, VariationalInference

In [4]:
# Load MNIST
data_path = "drive/MyDrive/dgm/data"
mnist = load_mnist(data_path=data_path, batch_size=64)

100%|██████████| 781/781 [00:00<00:00, 5096.30it/s]


In [5]:
!pip install tensorboard
!pip uninstall jax jaxlib -y
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: jax 0.3.10
Uninstalling jax-0.3.10:
  Successfully uninstalled jax-0.3.10
Found existing installation: jaxlib 0.3.10+cuda11.cudnn805
Uninstalling jaxlib-0.3.10+cuda11.cudnn805:
  Successfully uninstalled jaxlib-0.3.10+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached jax-0.3.10-py3-none-any.whl
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.3.10%2Bcuda11.cudnn805-cp37-none-manylinux2014_x86_64.whl (175.7 MB)


In [6]:
import os

experiment_name = 'mnist_vae_v2'
os.makedirs(f"drive/MyDrive/dgm/models/{experiment_name}", exist_ok=True)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# VAE and data parameters
z_dim = 32
batch_size = 64
image_shape = (28, 28)


# Define VAE and inference object
VAE = VariationalAutoEncoder(image_shape, z_dim).to(device)
VI = VariationalInference(beta=1)

# Setup optimizer
optimizer = torch.optim.Adam(VAE.parameters(), lr=1e-3)

# Run training
train(mnist, VAE, VI, optimizer, epochs=1000, device=device,
      val_every_epoch=20, checkpoint_every=20,
      tensorboard_logdir='logs', experiment_name=experiment_name)

# Save model
VAE.to(torch.device('cpu'))
torch.save(VAE, f"drive/MyDrive/dgm/models/{experiment_name}/final.pth")



Training ELBO: -0.092 	| 	 Validation ELBO: -0.056 | Progress:   0%|          | 4/1000 [00:33<2:13:48,  8.06s/it]